## Get Samseg v6.1 data (via THINQ v.1.0.0-rc.11)
```
mkdir -p /home/paul/cmet/data/20200609-mclaren-1.0.0-rc.11-42-g8d976b0--take4
cd /home/paul/cmet/data/20200609-mclaren-1.0.0-rc.11-42-g8d976b0--take4
aws s3 cp s3://cmet-scratch/maclaren-cmeds/demographics.tsv .
aws s3 cp \
  --recursive \
  --exclude "*" \
  --include "*subject_info.json" \
  --include "*.pdf" \
  s3://cmet-scratch/20200609-mclaren-1.0.0-rc.11-42-g8d976b0--take4/maclaren-cmeds/ .
find . -type d -name 'cache' -exec rm -rf {} \;
```

Get rid of cached `subject_info.json` files:

```
find . -type d -name 'cache' -exec rm -rf {} \;
```

## Get FreeSurfer v6.1 data 
```
mkdir -p /home/paul/cmet/data/20200714-maclaren-fs6/
cd /home/paul/cmet/data/20200714-maclaren-fs6/
aws s3 cp s3://cmet-scratch/maclaren-cmeds/demographics.tsv .
aws s3 cp \
  --recursive \
  --exclude "*" \
  --include "*.stats" \
  s3://cmet-scratch/20200714-maclaren-fs6/ .
```

## Get v7.1 data (both samseg and aseg)
```
mkdir -p /home/paul/cmet/data/20201006-maclaren-fs-7.1-samseg-aseg-long/
cd /home/paul/cmet/data/20201006-maclaren-fs-7.1-samseg-aseg-long/
aws s3 cp s3://cmet-scratch/maclaren-cmeds/demographics.tsv .
aws s3 cp \
  --recursive \
  --exclude "*" \
  --include "*.stats" \
  s3://cmet-scratch/20201006-maclaren-fs-7.1-samseg-aseg-long/ .
```

### Split data into seperate subdirs

To faciliate recursive processing of *.stats files
```
cd /home/paul/cmet/data/20201006-maclaren-fs-7.1-samseg-aseg-long/
mkdir cross
mkdir long
mkdir long-base
mv sub-??_run-?? ./cross/
mv sub-??_base ./long-base
mv sub* ./long/
```

### Rename the long dirs
This is gross
```
cd ./long
for DIR_STEM in `ls -1|sed 's/\..*//'`; do  SUB_NUM=`echo $DIR_STEM|sed 's/sub-//'|sed 's/_run.*//'`; mv ${DIR_STEM}.long.sub-${SUB_NUM}_base ${DIR_STEM}; done
```

# Get Samseg v? data from container `a30d4ca`

Which corresponds to git commitID `??todo??`

```
mkdir -p /home/paul/cmet/data/20201104-maclaren-rethinq-a30d4ca
cd /home/paul/cmet/data/20201104-maclaren-rethinq-a30d4ca
aws s3 cp s3://cmet-scratch/maclaren-cmeds/demographics.tsv .
aws s3 cp \
  --recursive \
  --exclude "*" \
  --include "*subject_info.json" \
  --include "*.pdf" \
  s3://cmet-scratch/20201104-maclaren-rethinq-a30d4ca/ .
```

Get rid of cached `subject_info.json` files:

```
find . -type d -name 'cache' -exec rm -rf {} \;
```

In [2]:
import json
import os
import fnmatch
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# imports find_json_files(); load_json_file(); load_dataset(); load_fs_dataset()
from cmeds import *
# imports calc_cvs(); session_permute(); monte_carlo_perm_test
from test_retest import *

In [3]:
structs_of_interest = [
    'Left-Lateral-Ventricle',
    'Left-Hippocampus',
    'Left-Amygdala',
    'Left-Caudate',
    'Left-Putamen',
    'Right-Lateral-Ventricle',
    'Right-Hippocampus',
    'Right-Amygdala',
    'Right-Caudate',
    'Right-Putamen'
]

fs61aseg_demofile = '/home/paul/cmet/data/20200714-maclaren-fs6/demographics.tsv'
fs61aseg_datadir = '/home/paul/cmet/data/20200714-maclaren-fs6/'

fs61samseg_demofile = '/home/paul/cmet/data/20200609-mclaren-1.0.0-rc.11-42-g8d976b0--take4/demographics.tsv'
fs61samseg_datadir = '/home/paul/cmet/data/20200609-mclaren-1.0.0-rc.11-42-g8d976b0--take4/'

fs71aseg_demofile = '/home/paul/cmet/data/20201006-maclaren-fs-7.1-samseg-aseg-long/demographics.tsv'
fs71aseg_datadir = '/home/paul/cmet/data/20201006-maclaren-fs-7.1-samseg-aseg-long/cross'

fs71samseg_demofile = '/home/paul/cmet/data/20201006-maclaren-fs-7.1-samseg-aseg-long/demographics.tsv'
fs71aseg_datadir = '/home/paul/cmet/data/20201006-maclaren-fs-7.1-samseg-aseg-long/cross'

rethinq_a30d4ca_demofile = '/home/paul/cmet/data/20201006-maclaren-fs-7.1-samseg-aseg-long/demographics.tsv'
rethinq_a30d4ca_datadir = '/home/paul/cmet/data/20201104-maclaren-rethinq-a30d4ca'

## Load data into pandas dataframes

In [15]:
maclaren_fs61aseg_df = load_fs_dataset(fs61aseg_datadir, fs61aseg_demofile, structs_of_interest);
maclaren_fs71aseg_df = load_fs_dataset(fs71aseg_datadir, fs61aseg_demofile, structs_of_interest);
maclaren_fs71samseg_df = load_fssamseg_dataset(fs71aseg_datadir, fs61aseg_demofile, structs_of_interest);

maclaren_rethinq_rc11_df, maclaren_fs61samseg_df_normative = \
  load_dataset(fs61samseg_datadir, fs61samseg_demofile, drop_subjects=[], vol_data_src='volume');

maclaren_rethinq_rc11_samseg6only_df, maclaren_fs61samseg_df_normative = \
  load_dataset(fs61samseg_datadir, fs61samseg_demofile, drop_subjects=[], vol_data_src='samseg_volume');

maclaren_rethinq_a30d4ca_df, maclaren_rethinq_a30d4ca_normative_df = \
  load_dataset(rethinq_a30d4ca_datadir, rethinq_a30d4ca_demofile, drop_subjects=[], vol_data_src='volume');

maclaren_rethinq_a30d4ca_samseg7only_df, maclaren_rethinq_a30d4ca_normative_df = \
  load_dataset(rethinq_a30d4ca_datadir, rethinq_a30d4ca_demofile, drop_subjects=[], vol_data_src='samseg_volume');

Dropping the following subjects []
Dropping the following subjects []
Dropping the following subjects []
Ignoring Subject (did it error out?) sub-01_run-39
Ignoring Subject (did it error out?) sub-01_run-02
Ignoring Subject (did it error out?) sub-01_run-09
Ignoring Subject (did it error out?) sub-01_run-08
Ignoring Subject (did it error out?) sub-01_run-24
Ignoring Subject (did it error out?) sub-01_run-33
Ignoring Subject (did it error out?) sub-01_run-13
Ignoring Subject (did it error out?) sub-01_run-16
Ignoring Subject (did it error out?) sub-01_run-14
Ignoring Subject (did it error out?) sub-01_run-32
Ignoring Subject (did it error out?) sub-01_run-01
Ignoring Subject (did it error out?) sub-01_run-36
Ignoring Subject (did it error out?) sub-01_run-06
Ignoring Subject (did it error out?) sub-01_run-26
Ignoring Subject (did it error out?) sub-01_run-27
Ignoring Subject (did it error out?) sub-01_run-40
Ignoring Subject (did it error out?) sub-01_run-03
Ignoring Subject (did it err

Ignoring Subject (did it error out?) sub-01_run-39
Ignoring Subject (did it error out?) rethinq
Ignoring Subject (did it error out?) sub-01_run-02
Ignoring Subject (did it error out?) rethinq
Ignoring Subject (did it error out?) sub-01_run-09
Ignoring Subject (did it error out?) rethinq
Ignoring Subject (did it error out?) sub-01_run-08
Ignoring Subject (did it error out?) rethinq
Ignoring Subject (did it error out?) sub-01_run-24
Ignoring Subject (did it error out?) rethinq
Ignoring Subject (did it error out?) sub-01_run-13
Ignoring Subject (did it error out?) rethinq
Ignoring Subject (did it error out?) sub-01_run-28
Ignoring Subject (did it error out?) rethinq
Ignoring Subject (did it error out?) sub-01_run-16
Ignoring Subject (did it error out?) rethinq
Ignoring Subject (did it error out?) sub-01_run-14
Ignoring Subject (did it error out?) rethinq
Ignoring Subject (did it error out?) sub-01_run-29
Ignoring Subject (did it error out?) rethinq
Ignoring Subject (did it error out?) sub

In [18]:
# Add some lateral regions together so we can directly compare to table 1 in MacLaren et al.
# https://www.nature.com/articles/sdata201437/tables/2
regions = [             
            [ ['Left-Hippocampus', 'Right-Hippocampus'],'Hippocampus' ],
            [ ['Left-Lateral-Ventricle', 'Right-Lateral-Ventricle'],'Lateral-Ventricles' ],
            [ ['Left-Amygdala', 'Right-Amygdala',],'Amygdala' ],
            [ ['Left-Putamen', 'Right-Putamen'],'Putamen' ],
            [ ['Left-Caudate', 'Right-Caudate'],'Caudate' ],
          ]

maclaren_fs61aseg_df = add_regions(maclaren_fs61aseg_df,regions)
maclaren_fs71aseg_df = add_regions(maclaren_fs71aseg_df,regions)
maclaren_fs71samseg_df = add_regions(maclaren_fs71samseg_df,regions)
maclaren_rethinq_rc11_df = add_regions(maclaren_rethinq_rc11_df,regions)
maclaren_rethinq_rc11_samseg6only_df = add_regions(maclaren_rethinq_rc11_samseg6only_df,regions)
maclaren_rethinq_a30d4ca_df = add_regions(maclaren_rethinq_a30d4ca_df,regions)
maclaren_rethinq_a30d4ca_samseg7only_df = add_regions(maclaren_rethinq_a30d4ca_samseg7only_df,regions)

In [19]:
# Setup for permutation tests

# Since samseg 6 has some issues with sub1, remove from all analyses
session_list= [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
subject_list= [2,3]

# The column name that holds session info in the demographics.tsv
session_col='session'
# The column name that holds subject info in the demographics.tsv
subject_col='subject_num'

# To match with the rows of table 1 in https://www.nature.com/articles/sdata201437/tables/2
structs_of_interest = ['Hippocampus', 'Lateral-Ventricles', 'Amygdala', 'Putamen', 'Caudate']

In [ ]:
# Run the permutation tests, this will take a while for large n
n = 10000

maclaren_covs_fs61aseg_df = monte_carlo_perm_test( \
  maclaren_fs61aseg_df, subject_list, session_list, subject_col, session_col, \
  structs_of_interest, n_itrs=n, method='gluer')

maclaren_covs_fs71aseg_df = monte_carlo_perm_test( \
  maclaren_fs71aseg_df, subject_list, session_list, subject_col, session_col, \
  structs_of_interest, n_itrs=n, method='gluer')

maclaren_covs_fs71samseg_df = monte_carlo_perm_test( \
  maclaren_fs71samseg_df, subject_list, session_list, subject_col, session_col, \
  structs_of_interest, n_itrs=n, method='gluer')

maclaren_covs_rethinq_rc11_df = monte_carlo_perm_test( \
  maclaren_rethinq_rc11_df, subject_list, session_list, subject_col, session_col, \
  structs_of_interest, n_itrs=n, method='gluer')

maclaren_covs_rethinq_rc11_samseg6only_df = monte_carlo_perm_test( \
  maclaren_rethinq_rc11_samseg6only_df, subject_list, session_list, subject_col, session_col, \
  structs_of_interest, n_itrs=n, method='gluer')

maclaren_covs_rethinq_a30d4ca_df = monte_carlo_perm_test( \
  maclaren_rethinq_a30d4ca_df, subject_list, session_list, subject_col, session_col, \
  structs_of_interest, n_itrs=n, method='gluer')

maclaren_covs_rethinq_a30d4ca_samseg7only_df = monte_carlo_perm_test( \
  maclaren_rethinq_a30d4ca_samseg7only_df, subject_list, session_list, subject_col, session_col, \
  structs_of_interest, n_itrs=n, method='gluer')

These tables are comparable to [Table 1 in MacLaren et al](https://www.nature.com/articles/sdata201437/tables/2)

In [22]:
maclaren_covs_fs61aseg_df 

,Hippocampus,Lateral-Ventricles,Amygdala,Putamen,Caudate
mean-vol,8522.698750,13563.502500,3424.332500,9855.987500,6863.805000
total-cov,2.053487,2.081874,3.230995,1.895118,1.706345
session-cov,2.258124,0.902939,3.159073,2.135972,1.557038
abs-diff-cov,0.204637,1.178935,0.071922,0.240854,0.149307
p-vals,0.200000,0.000000,0.700000,0.300000,0.200000


In [23]:
maclaren_covs_fs71aseg_df 

,Hippocampus,Lateral-Ventricles,Amygdala,Putamen,Caudate
mean-vol,8590.835000,13670.435000,3510.177500,9883.452500,6867.283750
total-cov,1.670197,1.910059,3.013569,1.429833,1.996576
session-cov,1.749787,0.728072,3.255942,1.496864,1.644559
abs-diff-cov,0.079589,1.181987,0.242373,0.067031,0.352017
p-vals,0.600000,0.000000,0.200000,0.400000,0.100000


In [24]:
maclaren_covs_fs71samseg_df 

,Hippocampus,Lateral-Ventricles,Amygdala,Putamen,Caudate
mean-vol,10770.732396,23208.657477,3779.890494,13517.240480,8726.351490
total-cov,1.389530,1.672704,1.556568,8.808640,2.079050
session-cov,1.069301,1.406157,1.675307,9.338202,2.004842
abs-diff-cov,0.320229,0.266547,0.118739,0.529562,0.074207
p-vals,0.000000,0.000000,0.000000,0.200000,0.700000


In [25]:
maclaren_covs_rethinq_rc11_df

,Hippocampus,Lateral-Ventricles,Amygdala,Putamen,Caudate
mean-vol,8660.343750,15643.992500,3290.566250,11229.761250,7051.126250
total-cov,0.732712,1.428306,1.213812,1.103016,0.928835
session-cov,0.754715,0.950413,1.216737,0.979911,0.789903
abs-diff-cov,0.022003,0.477893,0.002925,0.123105,0.138931
p-vals,0.900000,0.000000,1.000000,0.000000,0.000000


In [26]:
maclaren_covs_rethinq_rc11_samseg6only_df

,Hippocampus,Lateral-Ventricles,Amygdala,Putamen,Caudate
mean-vol,9355.625000,17344.837500,3414.012500,11434.600000,7083.587500
total-cov,0.780972,1.450123,1.155101,1.068060,1.106091
session-cov,0.827046,1.177534,1.156447,1.027789,0.942654
abs-diff-cov,0.046074,0.272589,0.001346,0.040271,0.163436
p-vals,0.100000,0.000000,1.000000,0.400000,0.000000


In [27]:
maclaren_covs_rethinq_a30d4ca_df

,Hippocampus,Lateral-Ventricles,Amygdala,Putamen,Caudate
mean-vol,8627.191250,15596.686250,3293.113750,11224.871250,7009.927500
total-cov,0.754608,1.434468,1.161294,1.152517,1.063610
session-cov,0.750160,0.927965,1.142108,1.036997,0.879061
abs-diff-cov,0.004448,0.506504,0.019186,0.115520,0.184549
p-vals,1.000000,0.000000,0.800000,0.200000,0.000000


In [28]:
maclaren_covs_rethinq_a30d4ca_samseg7only_df

,Hippocampus,Lateral-Ventricles,Amygdala,Putamen,Caudate
mean-vol,9312.000000,17266.912500,3415.975000,11579.137500,7023.587500
total-cov,0.807860,1.454232,1.145045,1.005290,1.264741
session-cov,0.846088,1.142637,1.147874,0.912069,1.052177
abs-diff-cov,0.038227,0.311595,0.002828,0.093222,0.212564
p-vals,0.600000,0.000000,1.000000,0.400000,0.000000
